In [1]:
import akshare as ak

bond_cb_redeem_jsl_df = ak.bond_cb_redeem_jsl()

/home/jovyan/data/.venv/lib/python3.12/site-packages/py_mini_racer/py_mini_racer.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [28]:
bonds_df = bond_cb_redeem_jsl_df.query(
    "强赎状态 not in ['已公告强赎', '已满足强赎条件', '公告不强赎'] and 强赎天计数 == '0/15 | 30'"
)
bonds_df

,代码,名称,现价,正股代码,正股名称,规模,剩余规模,转股起始日,最后交易日,到期日,转股价,强赎触发比,强赎触发价,正股价,强赎价,强赎天计数,强赎条款,强赎状态
77,123210,信服转债,150.300,300454,深信服,12.148,12.146,2024-02-02,NaT,2029-07-27,110.91,130,144.18,129.50,NaN,0/15 | 30,如果公司股票在任意连续三十个交易日中至少十五个交易日的收盘价格不低于当期转股价格的 130%...,
78,123199,山河转债,129.029,300452,山河药辅,3.200,3.198,2023-12-18,NaT,2029-06-12,13.30,130,17.29,13.33,NaN,0/15 | 30,如果公司股票在任意连续三十个交易日中至少十五个交易日的收盘价格不低于当期转股价格的130%（...,
79,123207,冠中转债,124.800,300948,冠中生态,4.000,3.997,2024-01-29,NaT,2029-07-21,10.44,130,13.57,10.97,NaN,0/15 | 30,如果公司股票连续三十个交易日中至少有十五个交易日的收盘价格不低于当期转股价格的 130%（含...,
80,110086,精工转债,117.068,600496,精工钢构,20.000,19.999,2022-10-28,NaT,2028-04-22,4.79,130,6.23,3.64,NaN,0/15 | 30,如果公司股票在任何连续三十个交易日中至少有十五个交易日的收盘价格不低于当期转股价格的 130...,
81,123195,蓝晓转02,129.951,300487,蓝晓科技,5.461,5.460,2023-10-23,NaT,2029-04-17,59.68,130,77.58,55.11,NaN,0/15 | 30,如果公司股票在任何连续三十个交易日中至少十五个交易日的收盘价格不低于当期转股价格的130%（...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,127110,广核转债,139.190,003816,中国广核,49.000,49.000,2026-01-15,NaT,2031-07-08,3.67,130,4.77,3.84,NaN,0/15 | 30,如果公司 A 股股票在任意连续三十个交易日中至少有十五个交易日的收盘价格不低于当期转股价格的...,
378,118058,微导转债,187.546,688147,微导纳米,11.700,11.700,2026-02-12,NaT,2031-08-05,33.57,130,43.64,42.39,NaN,0/15 | 30,如果公司股票在任何连续三十个交易日中至少十五个交易日的收盘价格不低于当期转股价格的 130%...,
379,113698,凯众转债,100.000,603037,凯众股份,3.080,3.080,2026-02-23,NaT,2031-08-14,12.70,130,16.51,13.91,NaN,0/15 | 30,如果公司股票在连续三十个交易日中至少十五个交易日的收盘价格不低于当期转股价格的 130%（含...,
380,127111,金威转债,100.000,002626,金达威,12.924,12.924,2026-02-26,NaT,2031-08-19,19.59,130,25.47,20.31,NaN,0/15 | 30,公司股票连续三十个交易日中至少有十五个交易日的收盘价不低于当期转股价格的 130%（含本数）；,


In [29]:
import backtrader as bt
import pandas as pd
import duckdb


available_bonds = tuple(bonds_df['代码'])

con = duckdb.connect(database=':memory:', read_only=False) 
path = '../dataset/data/convertible_bonds/**/data.parquet'
date = '2025-08-28'

query = f"""
        SELECT 
            *,
            dblow + curr_iss_amt AS rank_indicator
        FROM  read_parquet('{path}')
        WHERE
            NOT bond_nm LIKE '%退%'
            AND NOT price_tips LIKE '%待上市%'
            AND price < 140
            AND premium_rt < 8
            AND rating_cd LIKE '%A%'
            AND qstatus = '00'
            AND update_date = '{date}'
            AND bond_id in {available_bonds}
        ORDER BY rank_indicator ASC
        LIMIT 10
"""

df_all = con.execute(query).df()
con.close()


df_all

,bond_id,bond_nm,bond_py,price,increase_rt,stock_id,stock_nm,stock_py,sprice,sincrease_rt,...,after_next_put_dt,icons,is_min_price,blocked,putting,force_redeem_price_tip,notes,update_date,year_month,rank_indicator
0,123049,维尔转债,wezz,123.71,0.77,300190,维尔利,wel,4.14,2.22,...,1,[],0,0,N,,None,2025-08-28,2025-08,138.959
1,128105,长集转债,cjzz,132.10,-0.82,002616,长青集团,cqjt,6.53,-2.97,...,1,[],0,0,N,,None,2025-08-28,2025-08,145.370
2,110059,浦发转债,pfzz,113.20,-0.14,600000,浦发银行,pfyh,13.86,0.80,...,0,"{""M"": """"}",0,0,N,,None,2025-08-28,2025-08,468.370
